## Pandas怎样实现groupby分组统计

groupby：先对数据分组，然后在每个分组上应用聚合函数、转换函数

本次演示：  
一、分组然后使用聚合函数做数据统计  
二、遍历groupby的结果理解执行流程  
三、实例分组探索天气数据  

In [ ]:
import pandas as pd
import numpy as np

In [ ]:
df = pd.DataFrame({'A': ['foo', 'bar', 'foo', 'bar', 'foo', 'bar', 'foo', 'foo'],
                   'B': ['one', 'one', 'two', 'three', 'two', 'two', 'one', 'three'],
                   'C': np.random.randn(8),
                   'D': np.random.randn(8)})

In [ ]:
df

### 一、分组然后使用聚合函数做数据统计

#### 1、单个列groupby，查询所有数据列的统计

In [ ]:
df.groupby('A').sum()

我们看到：
1. groupby中的'A'变成了数据的索引列
2. 因为要统计sum，但B列不是数字，所以被自动忽略掉

#### 2、多个列groupby，查询所有数据列的统计

In [ ]:
df.groupby(['A','B']).mean()

我们看到：('A','B')成对变成了二级索引

#### 3、同时查看多种数据统计

In [ ]:
df.groupby('A').agg([np.sum, np.mean, np.std])

我们看到：列变成了多级索引

#### 4、查看单列的结果数据统计

In [ ]:
# 方法1：预过滤，性能更好
df.groupby('A')['C'].agg([np.sum, np.mean, np.std])

In [ ]:
# 方法2
df.groupby('A').agg([np.sum, np.mean, np.std])['C']

#### 5、不同列使用不同的聚合函数

In [ ]:
df.groupby('A').agg({"C":np.sum, "D":np.mean})

### 二、遍历groupby的结果理解执行流程

for循环可以直接遍历每个group

##### 1、遍历单个列聚合的分组

In [ ]:
for name,group in df.groupby('A'):
    print(name)
    print(group)
    print()

##### 2、遍历多个列聚合的分组

In [ ]:
for name,group in df.groupby(['A', 'B']):
    print(name)
    print(group)
    print()

可以看到，name是一个2个元素的tuple，代表不同的列

#### 3、做分组后的列筛选，得到一个Series

In [ ]:
df.groupby(['A', 'B'])['C']

In [ ]:
for name, group in df.groupby('A')['C']:
    print(name)
    print(group)
    print(type(group))
    print()

其实所有的聚合统计，都是在dataframe和series上进行的；

### 三、实例分组探索天气数据

In [ ]:
fpath = "./datas/beijing_tianqi/beijing_tianqi_2018.csv"
df = pd.read_csv(fpath)
# 替换掉温度的后缀℃
df.loc[:, "bWendu"] = df["bWendu"].str.replace("℃", "").astype('int32')
df.loc[:, "yWendu"] = df["yWendu"].str.replace("℃", "").astype('int32')
df.head()

#### 1、查看每个月的最高温度、最低温度、平均空气质量指数

In [ ]:
# 新增一列为月份
df['month'] = df['ymd'].str[:7]

In [ ]:
df.head()

In [ ]:
df.groupby('month').agg({"bWendu":np.max, "yWendu":np.min, "aqi":np.mean})

In [ ]:
# 加上这一句，能在jupyter notebook展示matplot图表
%matplotlib inline

In [ ]:
df.groupby('month').agg({"bWendu":np.max, "yWendu":np.min, "aqi":np.mean}).plot()

#### 2、查看每个月的最差空气质量

In [ ]:
data = df.groupby('month')['aqi'].max()
data

In [ ]:
data.plot()